<a href="https://colab.research.google.com/github/egorvkid/ml_algos/blob/master/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Russian Text classification using pymorphy2


[RuCode] FakeNews classification competition https://competitions.codalab.org/competitions/26284
score: 0.9585516913 (f1_score)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from time import time
import spacy
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.layers import Dense, Dropout,BatchNormalization, Activation
from keras.models import Sequential
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/My Drive/FakeNews/'
pd.set_option("display.max_colwidth", -1)
df = pd.read_csv(PATH + 'train.txt',sep = '\t')
df_test = pd.read_csv(PATH + 'test.txt', sep = '\t')
#df = pd.read_csv(PATH + 'train.csv', quotechar=None, quoting=3,sep='#')
#df_test = pd.read_csv(PATH + 'test.csv', quotechar=None, quoting=3)
df.head(1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,text,class
0,В Синьцзян-Уйгурском автономном районе Китая СУАР уйгурам будут давать по 10 тысяч юаней более 162 тысячи долларов за брак с представителями титульной нации ханьцами. Местные власти сообщили об этом на своем официальном сайте передает South China Morning Post.Из-за процветающих в районе сепаратистских настроений Синьцзян давно является головной болью для Пекина. Снять напряженность и укрепить связи между местным населением и ханьцами власти Китая планируют с помощью программы поддержки межэтнических браков.Как отмечает издание для СУАР 10 тысяч юаней это большие деньги учитывая что средний годовой доход на душу населения составляет всего 75 тысяч юаней более 12 тысячи долларов.Материалы по теме1117 13 июня 2014Великая антитеррористическая стенаНа борьбу с внутренними врагами КНР тратит больше средств чем на оборону страныПомимо этого смешанные пары за каждый год прожитый в гармонии друг с другом будут получать по 50 тысяч юаней более 8 тысяч долларов. Дети рожденные в браке между уйгурами и ханьцами смогут бесплатно посещать детский сад и учиться в средней школе. При поступлении ребенка в институт правительство выделит семье грант в 5 тысяч юаней 814 долларов. Супруги также получат субсидии на здравоохранение.Однако несмотря на всевозможные льготы с момента появления закона в начале августа этого года ни один брак пока не был зарегистрирован. Всего по информации газеты на текущий момент заключено 54 брака между уйгурами и ханьцами. Однако эти пары никаких льгот не получают. К тому же браки между уйгурами и ханьцами как сообщает издание обычно заканчиваются разводом.Уйгуры составляют более 72 процентов населения СУАР китайцы-ханьцы менее 26 процентов.,true


In [ ]:
df.dropna(inplace=True)
text = df['text']
text_test = df_test['text']
labels = df['class']
text

0        В Синьцзян-Уйгурском автономном районе Китая СУАР уйгурам будут давать по 10 тысяч юаней более 162 тысячи долларов за брак с представителями титульной нации  ханьцами. Местные власти сообщили об этом на своем официальном сайте передает South China Morning Post.Из-за процветающих в районе сепаратистских настроений Синьцзян давно является головной болью для Пекина. Снять напряженность и укрепить связи между местным населением и ханьцами власти Китая планируют с помощью программы поддержки межэтнических браков.Как отмечает издание для СУАР 10 тысяч юаней  это большие деньги учитывая что средний годовой доход на душу населения составляет всего 75 тысяч юаней более 12 тысячи долларов.Материалы по теме1117 13 июня 2014Великая антитеррористическая  стенаНа борьбу с внутренними врагами КНР тратит больше средств чем на оборону страныПомимо этого смешанные пары за каждый год прожитый в гармонии друг с другом будут получать по 50 тысяч юаней более 8 тысяч долларов. Дети рожденные в браке

In [ ]:
text.dropna(inplace=True)
text_test.dropna(inplace=True)
text_test

0       Британское министерство информации приняло декрет который запрещает СМИ без согласования с упоминаемыми персонами употреблять в новостях гендерно-некорректные слова включая мужчина и женщина.\nПо мнению главы ведомства невозможно точно определить пол без согласования с конкретной персоной. Даже наличие половых признаков не может быть убедительным так как человек сам определяет свою принадлежность к определённому полу.\nВсего в списке 16 слов которые могут прямо или неявно указывать на гендер человека. За употребление каждого из них без письменного согласия упоминаемого предусмотрены штрафы в разных объёмах. Привлекать к ответственности издания будут даже в том случае если у того кто был упомянут в контексте определённого гендера нет претензий к изданию.\n                                                                                                                                                                                                                                   

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.7MB/s 
     |████████████████████████████████| 8.0MB 5.9MB/s 


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

INFO - 20:25:20: Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts_ru/data
INFO - 20:25:20: format: 2.4, revision: 404381, updated: 2017-08-28T16:38:19.536338


In [ ]:
import re
from tqdm.notebook import tqdm
tqdm.pandas()
def tokenize(doc):
  words = []
  lst = re.split('\.|\s',doc)
  for word in lst:
    p = morph.parse(word)[0]
    words.append(p.normal_form)
  return words
tokens = text.progress_apply(tokenize)


In [ ]:
tokens_test = text_test.progress_apply(tokenize)

In [ ]:

nltk.download('stopwords')
stop_words = stopwords.words('russian')
def remove_stopwords(text):
    """
    Removing stopwords belonging to russian language
    
    """
    words = [w for w in text if w not in stop_words]
    return words


text = tokens.progress_apply(lambda x : remove_stopwords(x))
text_test = tokens_test.progress_apply(lambda x : remove_stopwords(x))
text.head(3)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0    [синьцзян-уйгурский, автономный, район, китай, суар, уйгур, давать, 10, тысяча, юань, 162, тысяча, доллар, брак, представитель, титульный, нация, , ханьцами, , местный, власть, сообщить, это, свой, официальный, сайт, передавать, south, china, morning, post, из-за, процветать, район, сепаратистский, настроение, синьцзян, давно, являться, головной, боль, пекин, , снять, напряжённость, укрепить, связь, местный, население, ханьцами, власть, китай, планировать, помощь, программа, поддержка, межэтнический, брак, отмечать, издание, суар, 10, тысяча, юань, , это, большой, деньга, учитывать, средний, годовой, доход, душа, население, составлять, 75, тысяча, юань, 12, тысяча, доллар, материал, теме1117, 13, июнь, 2014великий, антитеррористический, , стенана, борьба, внутренний, враг, кнр, тратить, большой, средство, оборона, страныпомить, это, ...]                                                                                                                                       
1    [режи

In [ ]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

text = text.apply(lambda x : combine_text(x))
text_test = text_test.apply(lambda x : combine_text(x))
text.head(3)

0    синьцзян-уйгурский автономный район китай суар уйгур давать 10 тысяча юань 162 тысяча доллар брак представитель титульный нация  ханьцами  местный власть сообщить это свой официальный сайт передавать south china morning post из-за процветать район сепаратистский настроение синьцзян давно являться головной боль пекин  снять напряжённость укрепить связь местный население ханьцами власть китай планировать помощь программа поддержка межэтнический брак отмечать издание суар 10 тысяча юань  это большой деньга учитывать средний годовой доход душа население составлять 75 тысяча юань 12 тысяча доллар материал теме1117 13 июнь 2014великий антитеррористический  стенана борьба внутренний враг кнр тратить большой средство оборона страныпомить это смешанный пара каждый год прожить гармония друг друг получать 50 тысяча юань 8 тысяча доллар  ребёнок родить брак уйгур ханьцами смочь бесплатно посещать детский сад учиться средний школа  поступление ребёнок институт правительство выделить семья гран

In [ ]:
text = pd.read_csv(PATH + 'clean_train.csv')
text_test = pd.read_csv(PATH + 'clean_test.csv')
#text.to_csv(PATH + 'clean_train.csv')
#text_test.to_csv(PATH + 'clean_test.csv')

In [ ]:
all_words = [word for tokens in text for word in tokens]
sentence_lengths = [len(tokens) for tokens in text]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))
MAX_SEQUENCE_LENGTH = 1375 #!!!!!!!!!!!!!!!!!!!!!!!!!!!

29915507 words total, with a vocabulary size of 302
Max sentence length is 11683


In [ ]:
labels = [1 if label=='true' else 0 for label in labels]

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

list_corpus = text.tolist()
list_labels = labels#.tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)

In [ ]:

clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

ValueError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
X_train_counts, count_vectorizer = cv(text)
X_test_counts = count_vectorizer.transform(text_test)
X_train_counts.shape

(23878, 132651)

In [ ]:
pd.Series(labels).value_counts()

true    19914
fake    3964 
Name: class, dtype: int64

In [ ]:
clf.fit(X_train_counts, labels)

y_predicted_counts = clf.predict(X_test_counts)

In [ ]:
#pred = ['true' if prediction else 'fake' for prediction in y_predicted_counts]
result = pd.DataFrame()
result['text'] = df_test['text']
result['class'] = y_predicted_counts
#result.to_csv(PATH + 'sub.txt',sep='\t',index = False)
result.to_csv(PATH + 'sub.csv',sep='\t',index = True)
result['class'].value_counts()

true    4975
fake    1049
Name: class, dtype: int64